# Avaliação Final Tecnologias Hacker - Análise de Logs Web

## Objetivo
O objetivo desse projeto é construir um código capaz de analisar se, de acordo com os logs, pode haver uma conexão suspeita no seu servidor web. Para isso, levamos em consideração que os logs disponibilizados estão em formato combined.

In [2]:
#bibliotecas
import re

## Pré Processamento

In [3]:
#Função que le o arquivo e retorna uma lista com as linhas do arquivo
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            return arquivo.readlines()
    except FileNotFoundError:
        print("Arquivo não encontrado")
        return None

In [6]:
lista_linhas_log = ler_arquivo('access.log')
lista_linhas_log[0]

'207.46.13.104 - - [08/Jul/2019:09:07:18 +0200] "GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1" 302 623 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"\n'

In [4]:
# Função de extração das informações
def extrai_info_log(lista_linhas):
    info_logs = {}
    for i in range(len(lista_linhas)):
        dic_log = {}
        info = lista_linhas[i].split()
        dic_log['ip'] = info[0]
        dic_log['id_cliente'] = info[1]
        dic_log['usuario'] = info[2]
        dic_log['data'] = info[3].replace('[','')
        dic_log['requisicao'] = info[5] + " " + info[6] + " " + info[7]
        dic_log['status'] = info[8]
        dic_log['bytes_resposta'] = info[9]
        dic_log['referenciador'] = info[10]
        dic_log['navegador'] = info[11].replace('"','')

        info_logs[i] = dic_log

    return info_logs




In [7]:
dicionariodelogs = extrai_info_log(lista_linhas_log)
dicionariodelogs[0]

{'ip': '207.46.13.104',
 'id_cliente': '-',
 'usuario': '-',
 'data': '08/Jul/2019:09:07:18',
 'requisicao': '"GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1"',
 'status': '302',
 'bytes_resposta': '623',
 'referenciador': '"-"',
 'navegador': 'Mozilla/5.0'}

In [8]:
#Função para separa somente os logs de get
def separa_logs_get(dicionario_logs):
    logs_get = {}
    for i in range(len(dicionario_logs)):
        if 'GET' in dicionario_logs[i]['requisicao']:
            logs_get[i] = dicionario_logs[i]
    return logs_get

#Função que separa os logs de post
def separa_logs_post(dicionario_logs):
    logs_post = {}
    for i in range(len(dicionario_logs)):
        if 'POST' in dicionario_logs[i]['requisicao']:
            logs_post[i] = dicionario_logs[i]
    return logs_post

In [9]:
dicionariodelogs_get = separa_logs_get(dicionariodelogs)
dicionariodelogs_post = separa_logs_post(dicionariodelogs)

### Analisando os logs contendo POST
Muitas vezes, pessoas má influenciadas tentam "injetar" no servidor muitas vezes isso acontece por meio de um "POST".

In [ ]:
def analisando_os_posts(log_post):
    logs_suspeitos = {}
    padroes_suspeitos = ['wp-login', 'wp-admin', 'xmlrpc.php', 'admin-ajax.php']

    for key, value in log_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao']:
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos


In [25]:
logs_post_suspeitos = analisando_os_posts(dicionariodelogs_post)
logs_post_suspeitos

{1111: {'ip': '178.128.116.50',
  'id_cliente': '-',
  'usuario': '-',
  'data': '11/Jul/2019:00:52:50',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 24062: {'ip': '89.108.87.179',
  'id_cliente': '-',
  'usuario': '-',
  'data': '22/Aug/2019:06:36:02',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '500',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26426: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /blog/xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '547',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26427: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '537',
  'referenciador'

### Analisando os log's contendo GET

In [10]:
def get_caminhos_suspeitos(info_logs):
    # Lista de caminhos considerados suspeitos
    caminhos_suspeitos = [
        '/admin',
        '/administrator',
        '/console',
        '/controlpanel',
        '/login',
        '/dashboard',
        '/dbadmin',
        '/server-status',
        '/private',
        '/phpmyadmin',

        'config',
        'config.php',
        'backup',
        'backup.sql',
        'debug',
        '.env',
        'settings.php',
        'db_backup.sql',
        'db_dump.sql',
        'htaccess',
        'htpasswd'

        'wp-login',
        'wp-admin',
        'wp-content',
        'wp-content/plugins',
        'wp-content/themes',
        'wp-content/uploads',
        'wp-includes',

        '/phpmyadmin',
        '/etc/passwd',
        'etc/shadow',
        '/proc/self/environ', 
        '/var/www/html',
        '/shell',
        '/index.php.bak',
        '/logs',
        '/robots.txt',
        '/wpad.dat',

    ]

    requisicoes_suspeitas = {}

    for key, log in info_logs.items():
        for caminho in caminhos_suspeitos:
            if caminho in log['requisicao']:
                requisicoes_suspeitas[key] = log
                break  # Parar após encontrar o primeiro caminho suspeito

    return requisicoes_suspeitas


get_caminhos_suspeitos(dicionariodelogs_get)

{2: {'ip': '194.160.223.18',
  'id_cliente': '-',
  'usuario': '-',
  'data': '08/Jul/2019:09:16:39',
  'requisicao': '"GET /wpad.dat HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '477',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 3: {'ip': '194.160.223.18',
  'id_cliente': '-',
  'usuario': '-',
  'data': '08/Jul/2019:09:16:39',
  'requisicao': '"GET /wpad.dat HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '477',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 4: {'ip': '194.160.223.18',
  'id_cliente': '-',
  'usuario': '-',
  'data': '08/Jul/2019:09:16:43',
  'requisicao': '"GET /wpad.dat HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '477',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 5: {'ip': '95.216.96.254',
  'id_cliente': '-',
  'usuario': '-',
  'data': '08/Jul/2019:09:21:59',
  'requisicao': '"GET /robots.txt HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '504',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 11

In [13]:
def ataque_de_forca_bruta(info_logs):

    from collections import defaultdict

    # Lista de caminhos considerados suspeitos
    caminhos_suspeitos = [
        '/login',
        'password',
        'admin'
    ]

    logs_suspeitos = {}

    requisicoes_por_corpo = defaultdict(list)

    for log in info_logs.values():
        if log['status'] != "200":
            requisicoes_por_corpo[log['requisicao']].append(log['status'])

    for key, log in info_logs.items():
        for caminho in caminhos_suspeitos:
            if caminho in log['requisicao']:
                if len(requisicoes_por_corpo[log['requisicao']]) > 5:
                    log['POSSÍVEL ATAQUE'] = "Ataque de forca bruta"
                    logs_suspeitos[key] = log


    return logs_suspeitos


ataque_de_forca_bruta(dicionariodelogs_post)

{1111: {'ip': '178.128.116.50',
  'id_cliente': '-',
  'usuario': '-',
  'data': '11/Jul/2019:00:52:50',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'POSSÍVEL ATAQUE': 'Ataque de forca bruta'},
 225233: {'ip': '45.90.216.84',
  'id_cliente': '-',
  'usuario': '-',
  'data': '17/Dec/2020:22:32:06',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'POSSÍVEL ATAQUE': 'Ataque de forca bruta'},
 267866: {'ip': '167.71.26.21',
  'id_cliente': '-',
  'usuario': '-',
  'data': '09/May/2021:05:16:57',
  'requisicao': '"POST //admin/vendor/phpunit/phpunit/src/Util/PHP/eval-stdin.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '629',
  'referenciador': '"https://www.google.com/"',
  'navegador': 'Mozilla/5.0',
  'POSSÍVEL ATAQUE': 'Ataque de forca bruta'},
 3

In [17]:
#DDos

def detecta_muitas_requisicoes_curto_intervalo(info_logs, limite=20, intervalo=5):
    from datetime import datetime
    from collections import defaultdict

    requisicoes_por_ip = defaultdict(list)
    for log in info_logs.values():
        data_hora = datetime.strptime(log['data'], "%d/%b/%Y:%H:%M:%S")
        requisicoes_por_ip[log['ip']].append(data_hora)

    ips_suspeitos = []
    for ip, requisicoes in requisicoes_por_ip.items():
        requisicoes.sort()
        for i in range(len(requisicoes) - limite + 1):
            if (requisicoes[i + limite - 1] - requisicoes[i]).total_seconds() <= intervalo:
                ips_suspeitos.append(ip)
                break

    return ips_suspeitos

detecta_muitas_requisicoes_curto_intervalo(dicionariodelogs_get)

['194.160.223.18',
 '77.247.181.162',
 '18.217.223.118',
 '125.64.94.213',
 '82.165.26.72',
 '88.198.60.25',
 '82.165.85.146',
 '160.153.156.143',
 '37.187.244.2',
 '91.183.83.65',
 '82.80.230.228',
 '92.118.37.64',
 '173.239.53.9',
 '185.234.217.32',
 '82.80.249.212',
 '144.76.25.220',
 '207.180.241.165',
 '82.80.249.156',
 '185.234.218.42',
 '178.143.41.16',
 '212.5.208.251',
 '82.80.249.159',
 '77.247.181.163',
 '90.176.20.42',
 '154.92.22.99',
 '45.154.255.73',
 '82.80.249.249',
 '82.80.249.200',
 '125.64.94.221',
 '178.20.55.18',
 '162.247.74.27',
 '125.64.94.214',
 '185.220.102.243',
 '185.220.102.249',
 '185.220.101.212',
 '167.71.26.21',
 '51.195.103.74',
 '109.88.66.20',
 '185.220.101.206',
 '92.118.24.40',
 '185.87.187.141',
 '211.91.147.2',
 '45.61.186.166',
 '119.91.129.52',
 '91.194.91.202',
 '206.81.15.229',
 '54.196.15.224',
 '13.229.126.223',
 '54.169.134.85',
 '46.188.17.17',
 '178.216.173.18',
 '109.88.42.205',
 '130.255.189.20',
 '84.224.109.179',
 '82.80.249.158',
 

In [13]:
#Directory Transversal
def detecta_exploracao_lfi(info_logs):
    lfi_suspeitas = {}
    for key, log in info_logs.items():
        if '..' in log['requisicao'] or '/etc/' in log['requisicao']:
            lfi_suspeitas[key] = log

    return lfi_suspeitas

detecta_exploracao_lfi(dicionariodelogs_get)

{34540: {'ip': '178.150.222.54',
  'id_cliente': '-',
  'usuario': '-',
  'data': '18/Sep/2019:15:55:39',
  'requisicao': '"GET //example.com/%2F.. HTTP/1.1"',
  'status': '404',
  'bytes_resposta': '467',
  'referenciador': '"http://szsbb.eu//example.com/%2F.."',
  'navegador': 'Mozilla/4.0'},
 37840: {'ip': '5.188.210.47',
  'id_cliente': '-',
  'usuario': '-',
  'data': '25/Sep/2019:03:00:59',
  'requisicao': '"GET /?_m=/etc/passwd&_c=triedni_ucitelia HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '590',
  'referenciador': '"http://szsbb.eu/?_m=/etc/passwd&_c=triedni_ucitelia"',
  'navegador': 'Mozilla/5.0'},
 37842: {'ip': '5.188.210.47',
  'id_cliente': '-',
  'usuario': '-',
  'data': '25/Sep/2019:03:01:00',
  'requisicao': '"GET /?_m=/etc/passwd%00&_c=triedni_ucitelia HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '596',
  'referenciador': '"http://szsbb.eu/?_m=/etc/passwd%00&_c=triedni_ucitelia"',
  'navegador': 'Mozilla/5.0'},
 37844: {'ip': '5.188.210.47',
  'id_clien

In [14]:
def detecta_gets_parametros_longos(info_logs, tamanho_limite=200):
    parametros_longos = {}
    for key, log in info_logs.items():
        if len(log['requisicao']) > tamanho_limite:
            parametros_longos[key] = log

    return parametros_longos

detecta_gets_parametros_longos(dicionariodelogs_get)

{56478: {'ip': '45.56.127.144',
  'id_cliente': '-',
  'usuario': '-',
  'data': '31/Oct/2019:21:07:31',
  'requisicao': '"GET /?_m=akcie&_c=exkurzia_budapest_2013%20or%20(1,2)=(select*from(select%20name_const(CHAR(111,108,111,108,111,115,104,101,114),1),name_const(CHAR(111,108,111,108,111,115,104,101,114),1))a)%20--%20and%201%3D1 HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '877',
  'referenciador': '"-"',
  'navegador': '-'},
 56479: {'ip': '45.56.127.144',
  'id_cliente': '-',
  'usuario': '-',
  'data': '31/Oct/2019:21:07:32',
  'requisicao': '"GET /?_m=akcie&_c=exkurzia_budapest_2013%27%20or%20(1,2)=(select*from(select%20name_const(CHAR(111,108,111,108,111,115,104,101,114),1),name_const(CHAR(111,108,111,108,111,115,104,101,114),1))a)%20--%20%27x%27=%27x HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '885',
  'referenciador': '"-"',
  'navegador': '-'},
 56480: {'ip': '45.56.127.144',
  'id_cliente': '-',
  'usuario': '-',
  'data': '31/Oct/2019:21:07:33',
  'requisicao':

In [16]:
def detecta_muitos_status_suspeitos(info_logs):
    from collections import defaultdict

    contagem_status_suspeito = defaultdict(int)
    requisicoes_status_suspeitos = {}
    status_suspeitos = ['404', '403', '401', '500']

    for key, log in info_logs.items():
        if log['status'] in status_suspeitos :
            contagem_status_suspeito[log['ip']] += 1
            requisicoes_status_suspeitos[key] = log

    return {ip: count for ip, count in contagem_status_suspeito.items() if count > 5}, requisicoes_status_suspeitos

detecta_muitos_status_suspeitos(dicionariodelogs_get)


({},
 {34540: {'ip': '178.150.222.54',
   'id_cliente': '-',
   'usuario': '-',
   'data': '18/Sep/2019:15:55:39',
   'requisicao': '"GET //example.com/%2F.. HTTP/1.1"',
   'status': '404',
   'bytes_resposta': '467',
   'referenciador': '"http://szsbb.eu//example.com/%2F.."',
   'navegador': 'Mozilla/4.0'}})

In [22]:
def detecta_acesso_direto_arquivos(info_logs):
    extensoes_sensitivas = ['.js', '.json', '.xml']

    acessos_diretos = {}
    for key, log in info_logs.items():
        for ext in extensoes_sensitivas:
            if ext in log['requisicao'].split(' ')[1]:
                acessos_diretos[key] = log
                break

    return acessos_diretos


detecta_acesso_direto_arquivos(dicionariodelogs_get)

{525: {'ip': '40.77.190.124',
  'id_cliente': '-',
  'usuario': '-',
  'data': '09/Jul/2019:18:36:49',
  'requisicao': '"GET /js/jquery.js HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '545',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 526: {'ip': '40.77.188.140',
  'id_cliente': '-',
  'usuario': '-',
  'data': '09/Jul/2019:18:36:50',
  'requisicao': '"GET /js/script.js HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '545',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 714: {'ip': '40.77.194.1',
  'id_cliente': '-',
  'usuario': '-',
  'data': '10/Jul/2019:04:16:42',
  'requisicao': '"GET /js/jquery.js HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '489',
  'referenciador': '"http://www.szsbb.eu/?_m=skola&_c=casopis_teoria_a_prax_farmaceuticky_laborant"',
  'navegador': 'Mozilla/5.0'},
 716: {'ip': '40.77.194.1',
  'id_cliente': '-',
  'usuario': '-',
  'data': '10/Jul/2019:04:16:45',
  'requisicao': '"GET /js/script.js HTTP/1.1"',
  'status': '30